Melika Ahmadi Ranjbar 97521036

---
Homework 4


#Question 3

In [73]:
import torch
import torchvision
from torchvision import datasets, models, transforms

# Load Dataset & Data Augmentation
transformTrain = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.RandomAffine(degrees=10, scale=(.9, 1.1), shear=0),
    #  transforms.RandomPerspective(distortion_scale=0.2),
     transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
     transforms.RandomRotation(30),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transformTest = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transformTrain)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transformTest)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ["plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


Files already downloaded and verified
Files already downloaded and verified


In [91]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.Conv2d(in_channels=6, out_channels=12, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, padding=1),
            nn.BatchNorm2d(20),
            nn.ReLU(),
            nn.Conv2d(in_channels=20, out_channels=36, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=36, out_channels=72, kernel_size=3, padding=2),
            nn.BatchNorm2d(72),
            nn.ReLU(),
            nn.Conv2d(in_channels=72, out_channels=120, kernel_size=3, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Flatten(),
            nn.Linear(120 * 6 * 6, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256, 10)
        )
        

    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # fc layer
        x = self.fc_layer(x)

        return x

    

In [96]:
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np

import torchvision
from torchvision import datasets, models, transforms

Model = Network()

Cuda = torch.cuda.is_available()
if Cuda:
    print('CUDA available, GPU started')
    Model = Network().cuda()
    Model = torch.nn.DataParallel(Model, device_ids=range(torch.cuda.device_count()))
else:
    print('No CUDA available, Continue With CPU')

Loss = nn.CrossEntropyLoss()
Optimizer = optim.Adam(Model.parameters(), lr=0.001, weight_decay=0.005)



for Epoch in range(2):
    running_loss = 0.0
    total_train = 0
    correct_train = 0
    train_accuracy = 0
    for i, data in enumerate(trainloader, start=0):
        # Input Data [Inputs, Labels]
        inputs, labels = data

        if Cuda:
            inputs = inputs.cuda()
            labels = labels.cuda()

        # Zero Gradient's Parameters
        Optimizer.zero_grad()

        # Forward, Backward, Optimize
        outputs = Model(inputs)
        loss = Loss(outputs, labels)
        loss.backward()
        Optimizer.step()

         # accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += predicted.eq(label.data).sum().item()
        train_accuracy = 100 * correct_train / total_train

        # Statics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                    (Epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    print('Epoch\'s Loss: ', running_loss)
    print('Epoch\'s Accuracy: ', train_accuracy)


CUDA available, GPU started
[1,  2000] loss: 2.242
[1,  4000] loss: 2.124
[1,  6000] loss: 2.080
[1,  8000] loss: 2.041
[1, 10000] loss: 2.017
[1, 12000] loss: 1.994
Epoch's Loss:  1000.3922016620636
Epoch's Accuracy:  13.046
[2,  2000] loss: 1.980
[2,  4000] loss: 1.984
[2,  6000] loss: 1.961
[2,  8000] loss: 1.959
[2, 10000] loss: 1.932
[2, 12000] loss: 1.955
Epoch's Loss:  958.9149264097214
Epoch's Accuracy:  12.966


In [97]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

total = 0
correct = 0
Model.eval()
# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        if Cuda:
            images = inputs.cuda()
            labels = labels.cuda()
        outputs = Model(images)
        _, predictions = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print('Total Accuracy: ', 100 * correct / total)

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Total Accuracy:  10.2
Accuracy for class plane is: 26.0 %
Accuracy for class car   is: 23.8 %
Accuracy for class bird  is: 0.0 %
Accuracy for class cat   is: 0.0 %
Accuracy for class deer  is: 0.0 %
Accuracy for class dog   is: 0.0 %
Accuracy for class frog  is: 27.1 %
Accuracy for class horse is: 0.0 %
Accuracy for class ship  is: 0.0 %
Accuracy for class truck is: 25.1 %
